In [30]:
import numpy as np
import pandas as pd
import warnings
import shutil
import os


import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.layers import LeakyReLU, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard



warnings.filterwarnings("ignore")

### Utils



In [31]:
def display_img(img_id):
    PATH_TO_IMGS = "CelebA_FR_Triplets/images/"

    img = mpimg.imread(PATH_TO_IMGS + img_id)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    


def preprocess_img(img):
    # Reshape the image to (100, 100, 3)
    img = tf.image.resize(img, [100, 100])
    # Rescale the image values to be between 0 and 1
    
    
    
    
    return (img - 122.5) / 122.5

### Images flow & Data Augmentation

In [32]:
train_datagen = ImageDataGenerator(
                        preprocessing_function=preprocess_img,
                        rotation_range = 20,
                        brightness_range = [0.6, 1],
                        zoom_range = 0.1,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        horizontal_flip=True,
                        
                    )

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_img)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_img)




train_generator = train_datagen.flow_from_directory("train/",
                                                    class_mode="binary",
                                                    target_size=(100, 100),
                                                    batch_size=32)

val_generator = val_datagen.flow_from_directory("validation/",
                                                class_mode="binary",
                                               target_size=(100, 100),
                                                batch_size=32)

test_generator = test_datagen.flow_from_directory("test/",
                                                  class_mode="binary",
                                                  target_size=(100, 100))

Found 40000 images belonging to 2 classes.
Found 3250 images belonging to 2 classes.
Found 3250 images belonging to 2 classes.


## Design Architecture of NN

In [33]:

class MLflowLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for metric, value in logs.items():
            mlflow.log_metric(metric, value, step=epoch)



checkpoint_filepath = 'checkpoints/nasnet_{epoch:02d}_{val_accuracy:04f}.h5'


model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Set to False to save the entire model
    save_best_only=True,     # Set to False to save the model after every epoch regardless of performance
    verbose=1                 # Logs a message whenever a model is saved
)


tensorboard = TensorBoard(log_dir="logs/", update_freq="batch")

In [34]:
nasnet_base = tf.keras.applications.NASNetMobile(weights="imagenet", include_top=False,input_shape=(100, 100, 3))




nasnet_base.trainable = False

# Add new layers on top of the model
x = nasnet_base.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x) 
x = Dropout(0.2)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)  # Add a Dense layer with a sigmoid activation for binary classification

# Create a new model
model = Model(inputs=nasnet_base.input, outputs=x)

# Compile the model



model.compile(optimizer=Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to verify the new architecture
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 stem_conv1 (Conv2D)         (None, 49, 49, 32)           864       ['input_13[0][0]']            
                                                                                                  
 stem_bn1 (BatchNormalizati  (None, 49, 49, 32)           128       ['stem_conv1[0][0]']          
 on)                                                                                              
                                                                                                  
 activation_2256 (Activatio  (None, 49, 49, 32)           0         ['stem_bn1[0][0]']     

                                                                     'separable_conv_2_bn_reductio
                                                                    n_right1_stem_1[0][0]']       
                                                                                                  
 separable_conv_1_reduction  (None, 25, 25, 11)           1920      ['separable_conv_1_pad_reducti
 _right2_stem_1 (SeparableC                                         on_right2_stem_1[0][0]']      
 onv2D)                                                                                           
                                                                                                  
 separable_conv_1_pad_reduc  (None, 53, 53, 32)           0         ['activation_2263[0][0]']     
 tion_right3_stem_1 (ZeroPa                                                                       
 dding2D)                                                                                         
          

 separable_conv_2_bn_reduct  (None, 25, 25, 11)           44        ['separable_conv_2_reduction_l
 ion_left4_stem_1 (BatchNor                                         eft4_stem_1[0][0]']           
 malization)                                                                                      
                                                                                                  
 reduction_right5_stem_1 (M  (None, 25, 25, 11)           0         ['reduction_pad_1_stem_1[0][0]
 axPooling2D)                                                       ']                            
                                                                                                  
 zero_padding2d_48 (ZeroPad  (None, 50, 50, 32)           0         ['adjust_relu_1_stem_2[0][0]']
 ding2D)                                                                                          
                                                                                                  
 reduction

 separable_conv_1_bn_reduct  (None, 13, 13, 22)           88        ['separable_conv_1_reduction_l
 ion_left1_stem_2 (BatchNor                                         eft1_stem_2[0][0]']           
 malization)                                                                                      
                                                                                                  
 separable_conv_1_bn_reduct  (None, 13, 13, 22)           88        ['separable_conv_1_reduction_r
 ion_right1_stem_2 (BatchNo                                         ight1_stem_2[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 activation_2269 (Activatio  (None, 13, 13, 22)           0         ['separable_conv_1_bn_reductio
 n)                                                                 n_left1_stem_2[0][0]']        
          

 _right2_stem_2 (SeparableC                                                                       
 onv2D)                                                                                           
                                                                                                  
 activation_2275 (Activatio  (None, 13, 13, 22)           0         ['separable_conv_1_bn_reductio
 n)                                                                 n_right3_stem_2[0][0]']       
                                                                                                  
 activation_2277 (Activatio  (None, 13, 13, 22)           0         ['separable_conv_1_bn_reductio
 n)                                                                 n_left4_stem_2[0][0]']        
                                                                                                  
 reduction_left2_stem_2 (Ma  (None, 13, 13, 22)           0         ['reduction_pad_1_stem_2[0][0]
 xPooling2

...

                                                                                                  
 normal_right4_10 (AverageP  (None, 4, 4, 176)            0         ['adjust_bn_10[0][0]']        
 ooling2D)                                                                                        
                                                                                                  
 separable_conv_2_bn_normal  (None, 4, 4, 176)            704       ['separable_conv_2_normal_left
 _left5_10 (BatchNormalizat                                         5_10[0][0]']                  
 ion)                                                                                             
                                                                                                  
 normal_add_1_10 (Add)       (None, 4, 4, 176)            0         ['separable_conv_2_bn_normal_l
                                                                    eft1_10[0][0]',               
          

 _left1_11 (BatchNormalizat                                         1_11[0][0]']                  
 ion)                                                                                             
                                                                                                  
 separable_conv_1_bn_normal  (None, 4, 4, 176)            704       ['separable_conv_1_normal_righ
 _right1_11 (BatchNormaliza                                         t1_11[0][0]']                 
 tion)                                                                                            
                                                                                                  
 separable_conv_1_bn_normal  (None, 4, 4, 176)            704       ['separable_conv_1_normal_left
 _left2_11 (BatchNormalizat                                         2_11[0][0]']                  
 ion)                                                                                             
          

 normal_add_2_11 (Add)       (None, 4, 4, 176)            0         ['separable_conv_2_bn_normal_l
                                                                    eft2_11[0][0]',               
                                                                     'separable_conv_2_bn_normal_r
                                                                    ight2_11[0][0]']              
                                                                                                  
 normal_add_3_11 (Add)       (None, 4, 4, 176)            0         ['normal_left3_11[0][0]',     
                                                                     'adjust_bn_11[0][0]']        
                                                                                                  
 normal_add_4_11 (Add)       (None, 4, 4, 176)            0         ['normal_left4_11[0][0]',     
                                                                     'normal_right4_11[0][0]']    
          

 tion)                                                                                            
                                                                                                  
 separable_conv_1_bn_normal  (None, 4, 4, 176)            704       ['separable_conv_1_normal_left
 _left5_12 (BatchNormalizat                                         5_12[0][0]']                  
 ion)                                                                                             
                                                                                                  
 activation_2434 (Activatio  (None, 4, 4, 176)            0         ['separable_conv_1_bn_normal_l
 n)                                                                 eft1_12[0][0]']               
                                                                                                  
 activation_2436 (Activatio  (None, 4, 4, 176)            0         ['separable_conv_1_bn_normal_r
 n)       

                                                                     'normal_bn_1_12[0][0]']      
                                                                                                  
 normal_concat_12 (Concaten  (None, 4, 4, 1056)           0         ['adjust_bn_12[0][0]',        
 ate)                                                                'normal_add_1_12[0][0]',     
                                                                     'normal_add_2_12[0][0]',     
                                                                     'normal_add_3_12[0][0]',     
                                                                     'normal_add_4_12[0][0]',     
                                                                     'normal_add_5_12[0][0]']     
                                                                                                  
 activation_2443 (Activatio  (None, 4, 4, 1056)           0         ['normal_concat_12[0][0]']    
 n)       

In [ ]:

model.fit( # test_generator,
            train_generator,
            validation_data=val_generator,
            epochs=4,
            batch_size=32,
           callbacks=[model_checkpoint_callback, tensorboard]
           )




Epoch 1/4
1250/1250 [==============================] - ETA: 0s - loss: 0.3988 - accuracy: 0.8248
Epoch 1: val_loss improved from inf to 0.34032, saving model to checkpoints/nasnet_01_0.847385.h5
1250/1250 [==============================] - 837s 654ms/step - loss: 0.3988 - accuracy: 0.8248 - val_loss: 0.3403 - val_accuracy: 0.8474
Epoch 2/4
1250/1250 [==============================] - ETA: 0s - loss: 0.3700 - accuracy: 0.8375
Epoch 2: val_loss improved from 0.34032 to 0.32305, saving model to checkpoints/nasnet_02_0.858462.h5
1250/1250 [==============================] - 861s 689ms/step - loss: 0.3700 - accuracy: 0.8375 - val_loss: 0.3230 - val_accuracy: 0.8585
Epoch 3/4
  51/1250 [>.............................] - ETA: 10:55 - loss: 0.3601 - accuracy: 0.8474

In [ ]:
model.save("nasnet_small_full_train.h5")